# Install

In [ ]:
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 224 kB 4.7 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
     |████████████████████████████████| 59 kB 1.1 MB/s 
     |████████████████████████████████| 285 kB 69.8 MB/s 
     |████████████████████████████████| 279 kB 72.9 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 62 kB 267 kB/s 
     |████████████████████████████████| 1.0 MB 52.7 MB/s 
     |████████████████████████████████| 132 kB 30.0 MB/s 
     |████████████████████████████████| 802 kB 47.4 MB/s 
     |████████████████████████████████| 41 kB 525 kB/s 
     |████████████████████████████████| 60.2 MB 1.1 MB/s 
     |████████████████████████████████| 1.6 MB 56.0 MB/s 
     |████████████████████████████████| 280 kB 54.2 MB/s 
     |████████████████████████████████| 148 kB 77.8 MB/s 
     |████████████████████████████████| 

# Tuning

In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# Path

In [ ]:
save_path = "/content/drive/MyDrive/colab/Data mining/Midterm exam/models-optimized"
training_dataset_path = "/content/drive/MyDrive/colab/Data mining/Midterm exam/refined_data.csv"
testing_dataset_path = "" #put a path to your test dataset in here

## Data

In [ ]:
#@title drop some column not importance
def read_dataset(path):
  try: cat_df = pd.read_csv(path, index_col="Unnamed: 0")
  except: cat_df = pd.read_csv(path)
  cat_df.head()
  cat_df["cfa_breed"] = cat_df["cfa_breed"].apply(lambda x: 1 if True else 0)
  cat_df["domestic_breed"] = cat_df["domestic_breed"].apply(lambda x: 1 if True else 0)
  cat_df = cat_df.drop(["domestic_breed", "cfa_breed", "breed2", "color1", "coat", "coat_pattern"], axis=1)
  return cat_df

In [ ]:
cat_df = read_dataset(training_dataset_path)
cat_df.info()

# Training

In [ ]:
label = "outcome_type"
X, y = cat_df.drop(label, axis=1), cat_df[label]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)


In [ ]:
#@title setting hyperparameter cases
import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'epochs': 25,  # number of training epochs (controls training time of NN models)
    'lr': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'emb_drop': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
    "bs" : ag.space.Categorical(32, 64)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

rf_options = {
    "n_estimators": 200,
    "criterion" : ag.space.Categorical("gini", "entropy"),
    "max_depth" : ag.space.Categorical(4, 8, 16)
}

xgb_options = {
    "max_depth" : ag.space.Int(lower=4, upper=10, default=4),
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  
                    # 'RF': rf_options,
                  #  "XGB" : xgb_options,
                   'GBM': gbm_options,
                   'FASTAI': nn_options,  
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 40  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}
metric = 'accuracy'

#runing
# train_data = pd.concat([X_train, y_train], axis=1)
# val_data   = pd.concat([X_test,  y_test ], axis=1)
train_data = pd.concat([X, y], axis=1)
# val_data   = pd.concat([X_test,  y_test ], axis=1)

predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(
    train_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

In [ ]:
y_pred = predictor.predict(X)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate_predictions(y_true=y, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                       model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        WeightedEnsemble_L2   0.779287       0.917298   8.429970                0.001115           1.650804            2       True         15
1                LightGBM/T8   0.777951       0.512815   3.491310                0.512815           3.491310            1       True          9
2                LightGBM/T7   0.776837       0.160210   2.362876                0.160210           2.362876            1       True          8
3                LightGBM/T1   0.774388       0.403368   3.287856                0.403368           3.287856            1       True          2
4                LightGBM/T2   0.773719       0.352296   2.929345                0.352296           2.929345            1       True          3
5                LightGBM/T3   0.772829       0.125968   3.739891         

In [ ]:
#@title best models
predictor.leaderboard(pd.concat([X, y], axis=1), silent=True)

In [ ]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [ ]:
#@title feature importance
predictor.feature_importance(pd.concat([X, y], axis=1))

These features in provided data are not utilized by the predictor and will be ignored: ['name']
Computing feature importance via permutation shuffling for 9 features using 5000 rows with 5 shuffle sets...
	59.16s	= Expected runtime (11.83s per shuffle set)
	36.81s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
sex_upon_outcome,0.26796,0.003018,1.930518e-09,5,0.274174,0.261746
outcome_hour,0.10548,0.003477,1.415363e-07,5,0.112640,0.098320
Cat/Kitten (outcome),0.06872,0.004696,2.600432e-06,5,0.078389,0.059051
outcome_month,0.05848,0.002365,3.201399e-07,5,0.063349,0.053611
outcome_weekday,0.05792,0.002598,4.848053e-07,5,0.063270,0.052570
dob_month,0.05148,0.001963,2.530221e-07,5,0.055521,0.047439
age_group,0.02872,0.001446,7.693069e-07,5,0.031698,0.025742
breed1,0.01344,0.001090,5.145389e-06,5,0.015684,0.011196
color2,0.01188,0.002119,1.165698e-04,5,0.016244,0.007516


# TEST WITH NEW DATA

In [ ]:
test_df = read_dataset(testing_dataset_path)
label = "outcome_type"
X_test, y_test = cat_df.drop(label, axis=1), cat_df[label]

In [ ]:
predictor = TabularPredictor.load(save_path) 
y_pred = predictor.predict(X_test)
y_pred[:20]

In [ ]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
